In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [ ]:
df = pd.read_csv('/content/IRIS.csv')

df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [ ]:
df.iloc[:, -1] = df.iloc[:, -1].\
                map({value: index for index, value in enumerate
                 (
                    df.iloc[:, -1].value_counts().index)}
                 )

df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,-0.900681,1.032057,-1.341272,-1.312977,0
1,-1.143017,-0.124958,-1.341272,-1.312977,0
2,-1.385353,0.337848,-1.398138,-1.312977,0
3,-1.506521,0.106445,-1.284407,-1.312977,0
4,-1.021849,1.263460,-1.341272,-1.312977,0


In [ ]:
# scaler = StandardScaler()
# X = scaler.fit_transform(df.iloc[:, :-1])
# X = pd.DataFrame(X, columns = df.columns[:-1])
# y = df.iloc[:, -1]

# df = pd.concat([X, y], axis = 1)

In [ ]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split\
                                  (
                                      X, y, test_size = 0.25, random_state = 42
                                  )

In [ ]:
print("X_train shape # {} ".format(X_train.shape))
print("X_test shape  # {} ".format(X_test.shape))
print("y_train shape # {} ".format(y_train.shape))
print("y_test shape  # {} ".format(y_test.shape))

X_train shape # (112, 4) 
X_test shape  # (38, 4) 
y_train shape # (112,) 
y_test shape  # (38,) 


In [ ]:
""" This script defines a PyTorch neural network model, Classifier,
    with specific layers and configurations. It also sets up logging
    and command-line argument parsing for model configuration.
"""

import torch
import torch.nn as nn
import logging
import argparse
from collections import OrderedDict

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    filename="model.log",
    filemode="w",
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)
logger = logging.getLogger(__name__)


class Classifier(nn.Module):
    """Defines a Classifier neural network with separate left, middle,
    right pathways and a fully connected layer.
    """

    def __init__(self):
        """Initializes the Classifier model with specific layer configurations."""
        super(Classifier, self).__init__()

        # Define three pathways with different configurations
        self.left = self._make_layers(layers_config=[(4, 32), (32, 16)], prefix="left")
        self.middle = self._make_layers(
            layers_config=[(4, 64), (64, 32), (32, 16)], prefix="middle"
        )
        self.right = self._make_layers(
            layers_config=[(4, 16), (16, 32), (32, 16)], prefix="right"
        )

        # Define a fully connected layer
        self.fc = self._connected_layer(
            layers_config=[(16 + 16 + 16, 32, 0.3), (32, 16, 0.7), (16, 3)], prefix="fc"
        )

    def _make_layers(self, layers_config: list, prefix: str):
        """Creates a sequence of layers based on the given configuration."""
        layers = OrderedDict()
        for idx, (input_features, output_features) in enumerate(layers_config):
            layers["{}-{}".format(prefix, idx)] = nn.Linear(
                in_features=input_features, out_features=output_features
            )
            layers[f"{prefix}-ReLU"] = nn.ReLU()

        return nn.Sequential(layers)

    def _connected_layer(self, layers_config: list, prefix: str):
        """Creates a fully connected layer with dropout based on the given configuration."""
        layers = OrderedDict()
        for idx, (input_features, output_features, dropout) in enumerate(
            layers_config[:-1]
        ):
            layers["{}-{}".format(prefix, idx)] = nn.Linear(
                in_features=input_features, out_features=output_features
            )
            layers[f"{prefix}-ReLU"] = nn.ReLU()
            layers[f"{prefix}-Dropout"] = nn.Dropout(p=dropout)

        # Last layer without dropout
        (input_features, output_features) = layers_config[-1]
        layers["out"] = nn.Linear(
            in_features=input_features, out_features=output_features
        )
        layers["softmax"] = nn.Softmax(dim=1)
        return nn.Sequential(layers)

    def forward(self, x):
        """Defines the forward pass of the model."""
        left = self.left(x)
        middle = self.middle(x)
        right = self.right(x)

        concat = torch.cat((left, middle, right), dim=1)
        output = self.fc(concat)

        return output

In [ ]:
model = Classifier()

In [ ]:
total_trainable_parameters = 0

for name, layer in model.named_parameters():
  print("Layer - {} & Parmaters - {} ".format(name, layer.numel()),'\n')
  total_trainable_parameters+=layer.numel()


print("_"*50, "\n\nTotal trainable parameters # {} ".format(total_trainable_parameters))

Layer - left.left-0.weight & Parmaters - 128  

Layer - left.left-0.bias & Parmaters - 32  

Layer - left.left-1.weight & Parmaters - 512  

Layer - left.left-1.bias & Parmaters - 16  

Layer - middle.middle-0.weight & Parmaters - 256  

Layer - middle.middle-0.bias & Parmaters - 64  

Layer - middle.middle-1.weight & Parmaters - 2048  

Layer - middle.middle-1.bias & Parmaters - 32  

Layer - middle.middle-2.weight & Parmaters - 512  

Layer - middle.middle-2.bias & Parmaters - 16  

Layer - right.right-0.weight & Parmaters - 64  

Layer - right.right-0.bias & Parmaters - 16  

Layer - right.right-1.weight & Parmaters - 512  

Layer - right.right-1.bias & Parmaters - 32  

Layer - right.right-2.weight & Parmaters - 512  

Layer - right.right-2.bias & Parmaters - 16  

Layer - fc.fc-0.weight & Parmaters - 1536  

Layer - fc.fc-0.bias & Parmaters - 32  

Layer - fc.fc-1.weight & Parmaters - 512  

Layer - fc.fc-1.bias & Parmaters - 16  

Layer - fc.out.weight & Parmaters - 48  

Layer -

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.AdamW(params = model.parameters(), lr = 0.001)

In [ ]:
X_train = torch.tensor(data = X_train, dtype = torch.float32)
X_test = torch.tensor(data = X_test, dtype = torch.float32)

train_loader = DataLoader(dataset = list(zip(X_train, y_train)),
                          batch_size = 16,
                          shuffle = True)

test_loader = DataLoader(dataset = list(zip(X_test, y_test)),
                        batch_size = 16,
                        shuffle = True)

In [ ]:
data, label = next(iter(train_loader))

print("With batch, data shape # {} ".format(data.shape),'\n')
print("With batch, label shape # {} ".format(label.shape),'\n')

With batch, data shape # torch.Size([16, 4])  

With batch, label shape # torch.Size([16])  



In [ ]:
EPOCHS = 100
history = {'loss': [], 'val_loss': [], 'accuracy': [], 'val_accuracy': []}
TRAIN_LOSS = []
VAL_LOSS   = []
TRAIN_ACCURACY = []
VAL_ACCURACY   = []

########################
#       Training       #
########################

# train the model
model.train()
# Run a loop with respect to defined Epoch
for epoch in range(EPOCHS):
  """
      1. Extract the data(X_batch), label(y_batch) from the `train_loader`
      2. Pass X_batch as a training data into the model and do the prediction
      3. Compute the Loss Function
      4. Store computed loss into TRAIN_LOSS
  """
  for (X_batch, y_batch) in train_loader:
    # print(epoch)
    # Do the prediction
    train_prediction = model(X_batch)
    # Compute the loss with the predicted and orginal
    train_loss = loss_function(train_prediction, y_batch)
    """
        1. Initiate the Optimizer
        2. Do the backward propagation with respect to train_loss
        3. Do the step with optimizer
    """
    # Initialize the optimizer
    optimizer.zero_grad()
    # Do back propagation
    train_loss.backward()
    # Do the step with respect to optimizer
    optimizer.step()

  ########################
  # Compute the Accuracy #
  ########################

  # Do the prediction of training
  train_predicted = torch.argmax(train_prediction, dim = 1)
  # Append the train accuracy
  TRAIN_ACCURACY.append(accuracy_score(train_predicted, y_batch))
  # Append the train loss
  history['accuracy'].append(accuracy_score(train_predicted, y_batch))
  with torch.no_grad():
    # Append the train loss
    TRAIN_LOSS.append(train_loss.item())
    # Append the train loss into the history
    history['loss'].append(train_loss.item())

  ########################
  #       Testing        #
  ########################

  """
      1. Extract the data(val_batch), label(val_batch) from the `test_loader`
      2. Pass val_batch as a training data into the model and do the prediction
      3. Compute the Loss Function
      4. Store computed loss into VAL_LOSS & VAL_ACCURACY
  """
  # Run a loop with respect to test_loader
  for (val_data, val_label) in test_loader:
    # print(val_data.shape)
    # Do the prediction
    test_prediction = model(val_data)
    # Compute the loss
    test_loss = loss_function(test_prediction, val_label)

  ##########################
  #  Compute the Accuracy  #
  ##########################

  # Append the test loss
  with torch.no_grad():
    VAL_LOSS.append(test_loss.item())
    history['val_loss'].append(test_loss.item())
    # Compute the accuracy
    test_predicted = torch.argmax(test_prediction, dim = 1)
    # Append the accuracy of testing data
    VAL_ACCURACY.append(accuracy_score(test_predicted, val_label))
    history['val_accuracy'].append(accuracy_score(test_predicted, val_label))

  #########################
  #        Display        #
  #########################

  print("Epoch {}/{} ".format(epoch + 1, EPOCHS))
  print("{}/{} [=========================] loss: {} - accuracy: {} - val_loss: {} - val_accuracy: {} ".format(train_loader.batch_size,\
                                                                                                              train_loader.batch_size,\
                                                                                                              np.array(train_loss.item()).mean(),
                                                                                                              accuracy_score(train_predicted, y_batch),\
                                                                                                              np.array(test_loss.item()).mean(),\
                                                                                                              accuracy_score(test_predicted, val_label)))


Epoch 1/100 
16/16 [=========================] loss: 1.1012606620788574 - accuracy: 0.4375 - val_loss: 1.0603574514389038 - val_accuracy: 0.8333333333333334 
Epoch 2/100 
16/16 [=========================] loss: 1.094594955444336 - accuracy: 0.5 - val_loss: 1.0641003847122192 - val_accuracy: 0.5 
Epoch 3/100 
16/16 [=========================] loss: 1.0569099187850952 - accuracy: 0.75 - val_loss: 1.0801936388015747 - val_accuracy: 0.5 
Epoch 4/100 
16/16 [=========================] loss: 1.0748481750488281 - accuracy: 0.4375 - val_loss: 0.9924845695495605 - val_accuracy: 0.8333333333333334 
Epoch 5/100 
16/16 [=========================] loss: 1.0599064826965332 - accuracy: 0.375 - val_loss: 1.0058015584945679 - val_accuracy: 0.6666666666666666 
Epoch 6/100 
16/16 [=========================] loss: 1.0034592151641846 - accuracy: 0.5 - val_loss: 0.9341297149658203 - val_accuracy: 0.6666666666666666 
Epoch 7/100 
16/16 [=========================] loss: 0.9573846459388733 - accuracy: 0.625 - 

In [ ]:
import argparse
import logging
import pandas as pd
from sklearn.model_selection import train_test_split



logging.basicConfig(
    level=logging.INFO,
    filename="dataset.log",
    filemode="w",
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)


class DataLoader:
    def __init__(self):
        pass

    def load_data(self, dataset="/content/IRIS.csv", split=0.20):
        """
        Load data from a CSV file and store it in the DataLoader instance.

        Parameters:
            dataset_path (str): The file path of the dataset to load.

        Returns:
            None
        """
        logging.info("Loading data")
        self.dataset = dataset
        self.split_ratio = split

        # Split the dataset
        self.split_dataset(dataset=self.dataset, split_ratio=self.split_ratio)

    def split_dataset(self, **dataset):
        """
        Split the loaded dataset into training and testing sets, and save them as CSV files.

        Raises:
            ValueError: If the dataset is not loaded.

        Returns:
            None
        """
        data_frame = pd.read_csv(dataset["dataset"])
        split_ratio = dataset["split_ratio"]

        logging.info("Splitting dataset")
        train, test = train_test_split(
            data_frame, test_size=split_ratio, random_state=42
        )
        try:
            train.to_csv(
                "/content/processed/train.csv",
                index=False,
            )
            test.to_csv(
                "/content/processed/test.csv",
                index=False,
            )
        except ValueError as e:
            logging.exception("File {} path is not found".title().format(e))


In [ ]:
loader = DataLoader()
loader.load_data(dataset = "/content/IRIS.csv", split = 0.25)

In [ ]:
import logging
import argparse
import os
import pandas as pd
import torch
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

logging.basicConfig(
    level=logging.INFO,
    filename="features.log",
    filemode="w",
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)
logger = logging.getLogger(__name__)

PATH = "/content/processed/"

class FeatureBuilder:
    """
    A class for building features from a given dataset.

    Attributes:
        None

    Methods:
        __init__: Initializes the FeatureBuilder instance.
        build_features: Builds features from a given dataset.
        _label_encoding: Performs label encoding on the 'species' column.
        _normalization: Performs standard scaling for normalization.

    Usage:
        feature_builder = FeatureBuilder()
        feature_builder.build_features(df)
    """

    def __init__(self):
        logger.info("Initializing FeatureBuilder")

    def build_features(self, df):
        """
        Builds features from a given dataset.

        Args:
            df (str): The path to the input CSV file.

        Returns:
            None
        """
        logging.info("Building Features starts")
        data_frame = pd.read_csv(df)
        data_frame.loc[:, "species"] = data_frame.loc[:, "species"].map(
            self._label_encoding(data_frame.loc[:, "species"])
        )
        # dataset = self._normalization(dataset=data_frame)

        logging.info("Creating the pickle file")
        joblib.dump(
            data_frame, "/content/processed/{}.pkl".format(
                df.split("/")[-1].split(".")[0]
            ),
        )

    def _label_encoding(self, target):
        """
        Performs label encoding on the 'species' column.

        Args:
            target (pd.Series): The target column to encode.

        Returns:
            dict: A dictionary mapping unique values to indices.
        """
        return {value: index for index, value in enumerate(target.value_counts().index)}

    def _normalization(self, dataset):
        """
        Performs standard scaling for normalization.

        Args:
            dataset (pd.DataFrame): The input dataset.

        Returns:
            pd.DataFrame: The normalized dataset.
        """
        logging.info("Standard scaling is used for normalization technique".title())

        scaler = StandardScaler()
        independent_features = scaler.fit_transform(
            dataset.loc[:, dataset.columns != "species"]
        )

        dependent_features = dataset.loc[:, "species"]
        independent_features = pd.DataFrame(independent_features)
        dependent_features = pd.DataFrame(dependent_features)
        return pd.concat([independent_features, dependent_features], axis=1)

    def create_data_loader(self):
        """
        Loads training and validation datasets from pickled files and converts them into tensors.

        This method performs the following steps:
        1. Loads the training dataset from 'train.pkl' and the validation dataset from 'test.pkl'.
        2. These datasets are expected to be in a predefined directory specified by the PATH variable.
        3. Each dataset is then converted into a tensor format suitable for model training and evaluation.
        4. The tensors are stored as 'train_loader' and 'test_loader' for training and validation datasets, respectively.

        The method does not return any values but updates the instance attributes related to data loaders.
        """
        train_dataset = joblib.load(filename=os.path.join(PATH, "train.pkl"))
        val_dataset = joblib.load(filename=os.path.join(PATH, "test.pkl"))
        print("hello")
        [
            self._convert_into_tensor(
                data=dataset, name="train_loader" if index == 0 else "test_loader"
            )
            for index, dataset in enumerate([train_dataset, val_dataset])
        ]

    def _convert_into_tensor(self, **dataset):
        """
        Converts a given dataset into a tensor and saves it as a pickle file.

        Parameters:
        - dataset (dict): A dictionary containing the dataset and its name.
                        The dataset is expected to have 'data' and 'name' keys.
                        'data' should be a pandas DataFrame with the last column as the target variable.
                        'name' is a string used for naming the output file.

        Steps:
        1. Splits the dataset into features (X) and the target variable (y).
        2. Converts the features (X) into a tensor of type torch.float32.
        3. Calls the '_tensor_to_dataloader' method to convert the tensor into a data loader format.
        4. Saves the data loader as a pickle file in the specified directory. The file name is derived from the dataset name.

        The method does not return any values but performs data processing and saving operations.
        """
        X, y = dataset["data"].iloc[:, :-1].values, dataset["data"].iloc[:, -1].values

        X = torch.tensor(data=X, dtype=torch.float32)

        loader = self._tensor_to_dataloader(X=X, y=y)

        joblib.dump(
            loader, os.path.join(PATH, "{}.pkl".format(dataset["name"]))
        )

    def _tensor_to_dataloader(self, **dataset):
        """
        Converts tensors of features and labels into a DataLoader object.

        This method is used to create a DataLoader from the given feature and label tensors.
        The DataLoader is a PyTorch utility that allows for efficient batching, shuffling,
        and loading of data during model training and evaluation.

        Parameters:
        - dataset (dict): A dictionary containing the features and labels.
                        It must have keys 'X' and 'y', where 'X' is a tensor of features
                        and 'y' is a tensor of labels.

        Returns:
        - data_loader (DataLoader): A DataLoader object created from the given tensors.
                                    It batches the data with a specified batch size (default is 64).

        The method takes the features and labels, zips them into a single dataset,
        and then creates a DataLoader object for efficient data handling.
        """
        data_loader = DataLoader(
            dataset=list(zip(dataset["X"], dataset["y"])), batch_size=16
        )
        return data_loader

In [ ]:
feature_builder = FeatureBuilder()
data_path = "/content/processed/"

[
    feature_builder.build_features(df=os.path.join(data_path, dataset))
    for dataset in ["train.csv", "test.csv"]
]

<ipython-input-116-66c6b193db1f>:53: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_frame.loc[:, "species"] = data_frame.loc[:, "species"].map(
<ipython-input-116-66c6b193db1f>:53: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_frame.loc[:, "species"] = data_frame.loc[:, "species"].map(


[None, None]

In [ ]:
feature_builder.create_data_loader()

hello


In [ ]:
train_check = joblib.load(filename = "/content/processed/train_loader.pkl")

In [ ]:
test_check = joblib.load(filename = "/content/processed/test_loader.pkl")

In [ ]:
for data, label in test_check:
  print(data, label)

tensor([[6.1000, 2.8000, 4.7000, 1.2000],
        [5.7000, 3.8000, 1.7000, 0.3000],
        [7.7000, 2.6000, 6.9000, 2.3000],
        [6.0000, 2.9000, 4.5000, 1.5000],
        [6.8000, 2.8000, 4.8000, 1.4000],
        [5.4000, 3.4000, 1.5000, 0.4000],
        [5.6000, 2.9000, 3.6000, 1.3000],
        [6.9000, 3.1000, 5.1000, 2.3000],
        [6.2000, 2.2000, 4.5000, 1.5000],
        [5.8000, 2.7000, 3.9000, 1.2000],
        [6.5000, 3.2000, 5.1000, 2.0000],
        [4.8000, 3.0000, 1.4000, 0.1000],
        [5.5000, 3.5000, 1.3000, 0.2000],
        [4.9000, 3.1000, 1.5000, 0.1000],
        [5.1000, 3.8000, 1.5000, 0.3000],
        [6.3000, 3.3000, 4.7000, 1.6000]]) tensor([2, 0, 1, 2, 2, 0, 2, 1, 2, 2, 1, 0, 0, 0, 0, 2])
tensor([[6.5000, 3.0000, 5.8000, 2.2000],
        [5.6000, 2.5000, 3.9000, 1.1000],
        [5.7000, 2.8000, 4.5000, 1.3000],
        [6.4000, 2.8000, 5.6000, 2.2000],
        [4.7000, 3.2000, 1.6000, 0.2000],
        [6.1000, 3.0000, 4.9000, 1.8000],
        [5.0000, 3

In [ ]:
X_train = torch.tensor(data = X_train, dtype = torch.float32)
X_test = torch.tensor(data = X_test, dtype = torch.float32)

train_loader = DataLoader(dataset = list(zip(X_train, y_train)),
                          batch_size = 16)

test_loader = DataLoader(dataset = list(zip(X_test, y_test)),
                        batch_size = 16)

In [ ]:
for data, label in test_loader:
  print(data, label)

tensor([[ 3.1100e-01, -5.8776e-01,  5.3530e-01,  1.7530e-03],
        [-1.7367e-01,  1.7263e+00, -1.1707e+00, -1.1815e+00],
        [ 2.2497e+00, -1.0506e+00,  1.7863e+00,  1.4480e+00],
        [ 1.8983e-01, -3.5636e-01,  4.2156e-01,  3.9617e-01],
        [ 1.1592e+00, -5.8776e-01,  5.9216e-01,  2.6470e-01],
        [-5.3718e-01,  8.0065e-01, -1.2844e+00, -1.0500e+00],
        [-2.9484e-01, -3.5636e-01, -9.0227e-02,  1.3323e-01],
        [ 1.2803e+00,  1.0645e-01,  7.6276e-01,  1.4480e+00],
        [ 4.3217e-01, -1.9762e+00,  4.2156e-01,  3.9617e-01],
        [-5.2506e-02, -8.1917e-01,  8.0370e-02,  1.7530e-03],
        [ 7.9567e-01,  3.3785e-01,  7.6276e-01,  1.0535e+00],
        [-1.2642e+00, -1.2496e-01, -1.3413e+00, -1.4444e+00],
        [-4.1601e-01,  1.0321e+00, -1.3981e+00, -1.3130e+00],
        [-1.1430e+00,  1.0645e-01, -1.2844e+00, -1.4444e+00],
        [-9.0068e-01,  1.7263e+00, -1.2844e+00, -1.1815e+00],
        [ 5.5333e-01,  5.6925e-01,  5.3530e-01,  5.2764e-01]]) tensor(

In [ ]:
for data, label in train_check:
  print(data, label)

tensor([[-1.0183,  1.3064, -1.3949, -1.3587],
        [-0.7730,  2.4639, -1.3370, -1.4927],
        [-0.0372, -0.7772,  0.7484,  0.9205],
        [ 0.2080,  0.8433,  0.4008,  0.5183],
        [ 1.0664,  0.1488,  0.5167,  0.3843],
        [-0.5277,  2.0009, -1.4528, -1.0905],
        [-0.5277,  1.5379, -1.3370, -1.3587],
        [-0.4051, -1.4717, -0.0626, -0.2861],
        [ 0.5759, -0.5457,  0.7484,  0.3843],
        [ 0.6986,  0.1488,  0.9801,  0.7865],
        [ 0.9438, -0.0827,  0.3429,  0.2502],
        [ 1.6796,  1.3064,  1.3277,  1.7249],
        [-0.1599, -0.3142,  0.2271,  0.1161],
        [ 2.1701, -0.0827,  1.6173,  1.1887],
        [-0.2825, -0.0827,  0.4008,  0.3843],
        [-0.8956,  1.0749, -1.3949, -1.3587]]) tensor([0, 0, 1, 2, 2, 0, 0, 2, 1, 1, 2, 1, 2, 1, 2, 0])
tensor([[ 2.2928, -0.5457,  1.6752,  1.0546],
        [-0.0372, -0.7772,  0.1691, -0.2861],
        [-0.7730,  0.8433, -1.3949, -1.3587],
        [-1.0183,  1.0749, -1.4528, -1.2246],
        [-0.8956,  1.7

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.AdamW(params = model.parameters(), lr = 0.001)

In [ ]:
train_data = joblib.load("/content/processed/train.pkl")
train_data

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.0,3.6,1.4,0.2,2
1,5.2,4.1,1.5,0.1,2
2,5.8,2.7,5.1,1.9,1
3,6.0,3.4,4.5,1.6,0
4,6.7,3.1,4.7,1.5,0
...,...,...,...,...,...
107,6.1,2.8,4.0,1.3,0
108,4.9,2.5,4.5,1.7,1
109,5.8,4.0,1.2,0.2,2
110,5.8,2.6,4.0,1.2,0


In [ ]:
X_train, y_train = train_data.iloc[:,:-1].values, train_data.iloc[:,-1].values

In [ ]:
test_data = joblib.load("/content/processed/test.pkl")
test_data

,sepal_length,sepal_width,petal_length,petal_width,species
0,6.1,2.8,4.7,1.2,2
1,5.7,3.8,1.7,0.3,0
2,7.7,2.6,6.9,2.3,1
3,6.0,2.9,4.5,1.5,2
4,6.8,2.8,4.8,1.4,2
5,5.4,3.4,1.5,0.4,0
6,5.6,2.9,3.6,1.3,2
7,6.9,3.1,5.1,2.3,1
8,6.2,2.2,4.5,1.5,2
9,5.8,2.7,3.9,1.2,2


In [ ]:
X_test, y_test = test_data.iloc[:,:-1].values, test_data.iloc[:,-1].values

In [ ]:

X_train = torch.tensor(data = X_train, dtype = torch.float32)
X_test = torch.tensor(data = X_test, dtype = torch.float32)

train_loader = DataLoader(dataset = list(zip(X_train, y_train)),
                          batch_size = 16,
                          shuffle = True)

test_loader = DataLoader(dataset = list(zip(X_test, y_test)),
                        batch_size = 16,
                        shuffle = True)

In [ ]:
"""
This module defines a Trainer class for training a neural network model on a given dataset.
It includes functionalities for training the model, computing loss and accuracy, performing
backward propagation, and saving historical data for analysis. The training process involves
both a training and a validation phase, with metrics logged for each.
"""

import logging
import argparse
import torch
import sys
import os
import numpy as np
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib

# Setting up logging configuration
logging.basicConfig(
    level=logging.INFO,
    filename="train_model.log",
    filemode="w",
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)

class Trainer:
    """
    The Trainer class encapsulates the functionality for training a machine learning model.
    It includes methods for handling the training process, computing loss and accuracy,
    backpropagation, and storing training history.
    """

    def __init__(self, epochs=200, lr=0.01, display=True):
        """
        Initializes the Trainer with specified epochs, learning rate, and display setting.
        Loads the training and testing datasets, initializes the model, loss function, and optimizer.
        Sets up a history dictionary to store training and validation loss and accuracy.
        """
        logging.info("Initialization is processing.".capitalize())

        self.epochs = epochs
        self.learning_rate = lr
        self.display = display
        self.train_loader = train_loader
        self.test_loader = filename = test_loader
        self.model = Classifier()

        self.loss_function = nn.CrossEntropyLoss()
        self.optimizer = optim.AdamW(params = self.model.parameters(), lr = self.learning_rate)
        self.history = {
            "train_loss": [],
            "train_accuracy": [],
            "val_loss": [],
            "val_accuracy": [],
        }

    def convert_to_long(self, label):
        """
        Converts label data to long tensor format, which is required for certain types of loss functions in PyTorch.
        """
        logging.info("Converting labels to long tensor format.".capitalize())

        return torch.Tensor(label).long()

    def _predict_and_evaluate_loss(self, dataset, specify):
        """
        Makes predictions on the given dataset and evaluates the loss.
        It also records the actual and predicted labels for accuracy computation.
        The 'specify' parameter determines whether to perform backpropagation ('train') or not ('test').
        """
        logging.info("Making predictions on the given dataset.".capitalize())

        actual = []
        predict = []
        loss_compute = []
        for data, label in dataset:
            label = self.convert_to_long(label=label)

            prediction = self.model(data)
            loss = self._compute_loss(prediction=prediction, label=label)

            if specify != "test":
                self._do_backward_propagation(loss=loss)

        actual.extend(label)
        predict.extend(torch.argmax(prediction, dim=1))
        loss_compute.append(loss.item())

        return actual, predict, np.array(loss_compute).mean()

    def _do_backward_propagation(self, loss):
        """
        Performs the backpropagation algorithm: resetting gradients, computing gradient, and updating model parameters.
        """
        logging.info("Performing backpropagation.".capitalize())

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def _compute_loss(self, prediction, label):
        """
        Computes the loss between the predicted and actual labels using the defined loss function.
        """
        logging.info("Computing loss.".capitalize())

        loss = self.loss_function(prediction, label)
        return loss

    def _compute_accuracy(self, actual, predict):
        """
        Computes the accuracy of predictions by comparing with actual labels.
        """
        logging.info("Computing accuracy.".capitalize())

        return accuracy_score(predict, actual)

    def _save_historical_data(self, loss, accuracy):
        """
        Saves the computed loss and accuracy in the training history for later analysis.
        """
        logging.info("Saving historical data.".capitalize())

        self.history["train_loss"].append(loss)
        self.history["train_accuracy"].append(accuracy)

    def _display(self, **data):
        """
        Displays the training progress and metrics if the display option is enabled.
        """
        logging.info("Displaying progress.".capitalize())

        print("Epochs - {}/{}".format(data["epoch"], data["total_epochs"]))
        print(
            "[===========] train_loss:{} - train_accuracy:{} - val_loss:{} - val_accuracy:{}".format(
                data["train_loss"],
                data["train_accuracy"],
                data["val_loss"],
                data["val_accuracy"],
            )
        )

    def train(self):
        """
        Executes the training process for the defined number of epochs.
        It involves training the model on the training dataset and validating it on the test dataset.
        """
        logging.info("Starting training process.".capitalize())

        for epoch in range(self.epochs):
            (
                train_actual,
                train_predict,
                train_total_loss,
            ) = self._predict_and_evaluate_loss(
                dataset=self.train_loader, specify="train"
            )

            (val_actual, val_predict, val_total_loss) = self._predict_and_evaluate_loss(
                dataset=self.test_loader, specify="test"
            )

            logging.info("Epoch: {}/{}".format(epoch + 1, self.epochs))

            train_accuracy = self._compute_accuracy(
                actual=train_actual, predict=train_predict
            )
            val_accuracy = self._compute_accuracy(
                actual=val_actual, predict=val_predict
            )

            logging.info("Saving the loss and train in the history".title())
            self._save_historical_data(loss=train_total_loss, accuracy=train_accuracy)
            self._save_historical_data(loss=val_total_loss, accuracy=val_accuracy)

            if self.display:
                self._display(
                    epoch=epoch,
                    total_epochs=self.epochs,
                    train_loss=train_total_loss,
                    train_accuracy=train_accuracy,
                    val_loss=val_total_loss,
                    val_accuracy=val_accuracy,
                )
            else:
                logging.info("Nothing is showing.".title())

In [ ]:
trainer = Trainer(epochs = 100, lr = 0.001)
trainer.train()

Epochs - 0/100
[===========] train_loss:1.0835645198822021 - train_accuracy:0.375 - val_loss:1.0632554292678833 - val_accuracy:0.5
Epochs - 1/100
[===========] train_loss:1.0940916538238525 - train_accuracy:0.1875 - val_loss:1.0778088569641113 - val_accuracy:0.3333333333333333
Epochs - 2/100
[===========] train_loss:1.0945425033569336 - train_accuracy:0.375 - val_loss:1.092178463935852 - val_accuracy:0.3333333333333333
Epochs - 3/100
[===========] train_loss:1.0390008687973022 - train_accuracy:0.4375 - val_loss:0.8651585578918457 - val_accuracy:0.8333333333333334
Epochs - 4/100
[===========] train_loss:0.8854857683181763 - train_accuracy:0.8125 - val_loss:1.0142838954925537 - val_accuracy:0.5
Epochs - 5/100
[===========] train_loss:0.9159165024757385 - train_accuracy:0.625 - val_loss:0.8789076209068298 - val_accuracy:0.8333333333333334
Epochs - 6/100
[===========] train_loss:0.7978383898735046 - train_accuracy:0.6875 - val_loss:0.8991393446922302 - val_accuracy:0.5
Epochs - 7/100
[====

In [ ]:
trained_model = trainer.model

In [ ]:
actual = []
predict_label = []

for data, label in test_loader:

  prediction = trained_model(data)
  prediction = torch.argmax(prediction, dim = 1)

  actual.extend(label)
  predict_label.extend(prediction)

print(len(actual))
print(len(predict_label))

38
38


In [ ]:
accuracy_score(predict_label, actual)

1.0

In [ ]:
precision_score(predict_label, actual, average = "macro")

0.8974747474747474

In [ ]:
recall_score(predict_label, actual, average = "macro")

0.8952991452991452

In [ ]:
f1_score(predict_label, actual, average = "macro")

0.8928571428571428